데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# 데이터 로드
data = pd.read_csv('gastric_cancer_data.csv')

# 데이터 탐색
print(data.head())
print(data.info())

# 결측치 처리: 평균으로 대체
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)

# 데이터프레임으로 변환
data = pd.DataFrame(data_imputed, columns=data.columns)

# 특성과 라벨 분리
X = data.drop('target', axis=1)  # 'target'은 위암 발생 여부
y = data['target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 정규화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("훈련 데이터 크기:", X_train.shape)
print("테스트 데이터 크기:", X_test.shape)


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# 데이터 로드
data = pd.read_csv('gastric_cancer_data.csv')

# 데이터 탐색
print(data.head())
print(data.info())
print(data.describe())

# 결측치 처리: 수치형 변수는 평균으로, 범주형 변수는 최빈값으로 대체
num_features = data.select_dtypes(include=['int64', 'float64']).columns
cat_features = data.select_dtypes(include=['object']).columns

# 수치형 변수 전처리
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# 범주형 변수 전처리
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 전체 전처리 파이프라인
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# 데이터 전처리
X = data.drop('target', axis=1)  # 'target'은 위암 발생 여부
y = data['target']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 전처리 적용
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

print("훈련 데이터 크기:", X_train.shape)
print("테스트 데이터 크기:", X_test.shape)




모델

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# 모델 정의
model = RandomForestClassifier(random_state=42)

# 하이퍼파라미터 튜닝을 위한 그리드 서치
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적의 모델
best_model = grid_search.best_estimator_
print("최적 하이퍼파라미터:", grid_search.best_params_)


모델 학습

In [ ]:
# 모델 학습
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# 학습 과정 시각화
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='훈련 정확도')
plt.plot(history.history['val_accuracy'], label='검증 정확도')
plt.title('정확도 변화')
plt.xlabel('에포크')
plt.ylabel('정확도')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='훈련 손실')
plt.plot(history.history['val_loss'], label='검증 손실')
plt.title('손실 변화')
plt.xlabel('에포크')
plt.ylabel('손실')
plt.legend()

plt.show()


신경망

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 모델 정의
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 요약
model.summary()


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 모델 정의
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류
])

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 요약
model.summary()


모델 학습 및 평가

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 모델 학습
best_model.fit(X_train, y_train)

# 예측
y_pred = best_model.predict(X_test)

# 성능 평가
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print("정확도:", accuracy)
print("혼동 행렬:\n", conf_matrix)
print("분류 보고서:\n", class_report)


In [ ]:
# 특성 중요도 분석을 위한 SHAP 값 계산
import shap

# SHAP 값 계산
explainer = shap.KernelExplainer(model.predict, X_train)
shap_values = explainer.shap_values(X_test)

# SHAP 값 시각화
shap.summary_plot(shap_values, X_test, feature_names=data.drop('target', axis=1).columns)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 중요 특성 시각화
feature_importances = best_model.feature_importances_
features = X.columns

# 중요도 데이터프레임 생성
importance_df = pd.DataFrame({'특성': features, '중요도': feature_importances})
importance_df = importance_df.sort_values(by='중요도', ascending=False)

# 중요 특성 바 차트
plt.figure(figsize=(10, 6))
sns.barplot(x='중요도', y='특성', data=importance_df)
plt.title('특성 중요도')
plt.show()

# 혼동 행렬 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['음성', '양성'], yticklabels=['음성', '양성'])
plt.ylabel('실제값')
plt.xlabel('예측값')
plt.title('혼동 행렬')
plt.show()


모델 최적화

In [ ]:
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터 튜닝을 위한 GridSearchCV 설정
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# RandomForestClassifier 예시
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# 최적의 모델
best_rf_model = grid_search.best_estimator_
print("최적 하이퍼파라미터:", grid_search.best_params_)

# 최적 모델 평가
best_rf_y_pred = best_rf_model.predict(X_test)
best_rf_conf_matrix = confusion_matrix(y_test, best_rf_y_pred)
print("최적 Random Forest 혼동 행렬:\n", best_rf_conf_matrix)


# 결론

In [ ]:
# 결론
- 위암 발생 예측 모델을 구축하기 위해 심층 신경망과 랜덤 포레스트 모델을 사용하여 성능을 평가했습니다.
- 두 모델 모두 양호한 성능을 보였으며, 향후 연구에서는 데이터 증강 및 추가적인 특성 엔지니어링을 통해 모델 성능을 개선할 수 있습니다.

# 향후 연구 방향
1. **데이터 증강**: 더 많은 데이터를 수집하거나 기존 데이터를 변형하여 모델의 일반화 능력을 향상시킬 수 있습니다.
2. **다양한 모델 시도**: XGBoost, LightGBM과 같은 다른 머신러닝 알고리즘을 적용해 볼 수 있습니다.
3. **특성 엔지니어링**: 도메인 지식을 바탕으로 새로운 특성을 생성하여 모델 성능을 향상시킬 수 있습니다.
4. **해석 가능성**: SHAP 및 LIME과 같은 기법을 통해 모델의 예측 결과를 해석하고, 의사결정에 활용할 수 있습니다.
